In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_esxdeploy_filter_distilbert_nonumber_unpair_2'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
# tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def read_passages(path_data_train):
    df_train = pd.read_csv(path_data_train)
    anchers_train = df_train['ancher'].to_list()
    documents_train = df_train['log'].to_list()
    labels_str_train = df_train['label'].to_list()

    samples_train = [' '.join([ancher, document]) for ancher, document in zip(anchers_train,documents_train)]
    
    labels_index = sorted(list(set(labels_str_train)))
    labels_all = {l:idx for idx, l in enumerate(labels_index)}
    labels_train = [labels_all[label_str] for label_str in labels_str_train]

    return samples_train, samples_train, labels_train, labels_train, labels_index
# end

In [5]:
# call the function
dir_data = 'datasource'
filename_train = 'esxdeploy_20220512_ancher_sample_nonumber_manual_edit_3.csv'
path_file_train = os.path.join(dir_data, filename_train)

train_samples, valid_samples, train_labels, valid_labels, target_names = read_passages(path_file_train)

In [6]:
# train_samples[837]

In [7]:
train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [8]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = SimpleDataset(train_encodings, train_labels)
valid_dataset = SimpleDataset(valid_encodings, valid_labels)

In [10]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
model = DistilBertForSequenceClassification.from_pretrained(model_info, num_labels=len(target_names))

load model from official


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    # print('labels: {}'.format(labels))
    # print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=10,
    metric_for_best_model='f1'
)

In [13]:
# trainer = Trainer(
#     model=model,                         # the instantiated Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     compute_metrics=compute_metrics,     # the callback that computes metrics of interest
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
# )

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [14]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6062
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 60620
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000500,0.148164,0.966678,0.965963,0.826690,0.881439
2,0.000800,0.080325,0.983834,0.968486,0.931248,0.948909
3,0.000100,0.095336,0.984329,0.953434,0.951201,0.952314
4,0.000600,0.050999,0.989607,0.974001,0.962304,0.968061
5,0.000200,0.031989,0.993896,0.984954,0.977783,0.981335
6,0.000200,0.031829,0.994226,0.985971,0.978785,0.982344
7,0.000100,0.029496,0.995051,0.993015,0.976778,0.984727
8,0.000000,0.015586,0.996041,0.987983,0.987983,0.987983
9,0.000100,0.019895,0.996536,0.989093,0.989895,0.989493
10,0.000000,0.005878,0.998515,0.998353,0.992623,0.995467


***** Running Evaluation *****
  Num examples = 6062
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-3031
Configuration saved in ./results/checkpoint-3031/config.json
Model weights saved in ./results/checkpoint-3031/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-33341] due to args.save_total_limit
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 6062
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-6062
Configuration saved in ./results/checkpoint-6062/config.json
Model weights saved in ./results/checkpoint-6062/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-36372] due to args.save_total_limit
<ipython-input-8-869644865237>:7: 

TrainOutput(global_step=60620, training_loss=0.051204487247961165, metrics={'train_runtime': 4453.7243, 'train_samples_per_second': 27.222, 'train_steps_per_second': 13.611, 'total_flos': 1.606034741305344e+16, 'train_loss': 0.051204487247961165, 'epoch': 20.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 6062
  Batch size = 2
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.0028657144866883755,
 'eval_accuracy': 0.9991751897063675,
 'eval_precision': 0.9995469372961218,
 'eval_recall': 0.9954462659380692,
 'eval_f1': 0.9974860832604912,
 'eval_runtime': 43.1152,
 'eval_samples_per_second': 140.6,
 'eval_steps_per_second': 70.3,
 'epoch': 20.0}

In [16]:
model.save_pretrained(model_dir)

Configuration saved in model_esxdeploy_filter_distilbert_nonumber_unpair_2/config.json
Model weights saved in model_esxdeploy_filter_distilbert_nonumber_unpair_2/pytorch_model.bin


In [17]:
import json
with open(f'{model_dir}/labels.json', 'w+') as file:
    file.write(json.dumps(target_names))